colab : 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization, Conv1D, MaxPooling1D, Dropout, Flatten, Activation
from sklearn.model_selection import KFold

In [ ]:
import pandas as pd
import numpy as np
from numpy import *
from sklearn import svm
from scipy.stats import randint, uniform
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import  GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report,roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline

train=pd.read_csv('/content/drive/MyDrive/월간데이콘11/train_features.csv')
train_labels=pd.read_csv('/content/drive/MyDrive/월간데이콘11/train_labels.csv')
test=pd.read_csv('/content/drive/MyDrive/월간데이콘11/test_features.csv')
submission=pd.read_csv('/content/drive/MyDrive/월간데이콘11/sample_submission.csv')

# 전처리


총 가속도 합을 변수로 추가하였습니다.

나름의 효과가 있어 긍정적으로 모델에 반영하였습니다.

In [ ]:
train["acc_total"]=(train["acc_x"]**2+train["acc_y"]**2+train["acc_z"]**2)**0.5
test["acc_total"]=(test["acc_x"]**2+test["acc_y"]**2+test["acc_z"]**2)**0.5

스케일링을 조금 더 쉽게 하기 위해서 임시로 데이터셋을 잘랐습니다.

In [ ]:
acc=train[["acc_x","acc_y","acc_z"]]
gy=train[["gy_x","gy_y","gy_z"]]

In [ ]:
acto=train["acc_total"]

스케일링을 하는 것이 결과면에서 조금 더 나은 것 같아 진행하였습니다.

In [ ]:
train["acc_x_norm"]=(train["acc_x"]-np.nanmean(acc))/np.nanstd(acc)
train["acc_y_norm"]=(train["acc_y"]-np.nanmean(acc))/np.nanstd(acc)
train["acc_z_norm"]=(train["acc_z"]-np.nanmean(acc))/np.nanstd(acc)

train["gy_x_norm"]=(train["gy_x"]-np.nanmean(gy))/np.nanstd(gy)
train["gy_y_norm"]=(train["gy_y"]-np.nanmean(gy))/np.nanstd(gy)
train["gy_z_norm"]=(train["gy_z"]-np.nanmean(gy))/np.nanstd(gy)

train["acc_total_norm"]=(train["acc_total"]-mean(acto))/std(acto)

In [ ]:
test["acc_x_norm"]=(test["acc_x"]-np.nanmean(acc))/np.nanstd(acc)
test["acc_y_norm"]=(test["acc_y"]-np.nanmean(acc))/np.nanstd(acc)
test["acc_z_norm"]=(test["acc_z"]-np.nanmean(acc))/np.nanstd(acc)

test["gy_x_norm"]=(test["gy_x"]-np.nanmean(gy))/np.nanstd(gy)
test["gy_y_norm"]=(test["gy_y"]-np.nanmean(gy))/np.nanstd(gy)
test["gy_z_norm"]=(test["gy_z"]-np.nanmean(gy))/np.nanstd(gy)

test["acc_total_norm"]=(test["acc_total"]-mean(acto))/std(acto)

원래의 데이터는 드랍하였습니다.

In [ ]:
train=train.drop(columns=["acc_x", "acc_y", "acc_z",
                   "gy_x", "gy_y", "gy_z", "acc_total"])

In [ ]:
test=test.drop(columns=["acc_x", "acc_y", "acc_z",
                   "gy_x", "gy_y", "gy_z", "acc_total"])

# data generate

데이터 불균형이 있긴 한데, 불균형 정도가 좀 심한 것 같아서 샘플링은 부적절하다 느꼈습니다.

차라리 데이터를 임의의 시간에서 잘라 붙여서, 데이터 자체의 크기를 늘려보았습니다.

In [ ]:
# data #
new_train=train

In [ ]:
import random
from tqdm import tqdm

for i in tqdm(range(3125), desc="tqdm example", mininterval=0.01):
    temp=train[train["id"]==i]
    aa=random.randrange(1,600)
    temp2=temp[aa:600].append(temp[0:aa])
    new_train=new_train.append(temp2)
    
for i in tqdm(range(3125), desc="tqdm example", mininterval=0.01):
    temp=train[train["id"]==i]
    aa=random.randrange(1,600)
    temp2=temp[aa:600].append(temp[0:aa])
    new_train=new_train.append(temp2)
    
for i in tqdm(range(3125), desc="tqdm example", mininterval=0.01):
    temp=train[train["id"]==i]
    aa=random.randrange(1,600)
    temp2=temp[aa:600].append(temp[0:aa])
    new_train=new_train.append(temp2)
    
for i in tqdm(range(3125), desc="tqdm example", mininterval=0.01):
    temp=train[train["id"]==i]
    aa=random.randrange(1,600)
    temp2=temp[aa:600].append(temp[0:aa])
    new_train=new_train.append(temp2)
    
for i in tqdm(range(3125), desc="tqdm example", mininterval=0.01):
    temp=train[train["id"]==i]
    aa=random.randrange(1,600)
    temp2=temp[aa:600].append(temp[0:aa])
    new_train=new_train.append(temp2)

In [ ]:
new_y=train_labels['label'].append(train_labels['label'])
new_y=new_y.append(train_labels['label'])
new_y=new_y.append(train_labels['label'])
new_y=new_y.append(train_labels['label'])
new_y=new_y.append(train_labels['label'])

X=tf.reshape(np.array(new_train.iloc[:,2:]),[-1, 600, 7])
X.shape

In [ ]:
y = tf.keras.utils.to_categorical(new_y) 
y.shape


# 1d cnn

In [ ]:
test_X=tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, 7])

모델링은 1d cnn을 활용했습니다.

논문을 참고했는데, 보통 cnn을 많이 쓰길래 시도해봤더니 1d cnn이 가장 좋은 결과가 나왔습니다.

깊은 모델, 얕은 모델, 그리고 그 중간. 이 세가지 cnn을 만들어 soft voting을 하면 결과가 좋게 나왔습니다.

repeat을 주는 대신 drop을 0.5로 설정하여 과적합을 최대한 피하고자 했습니다. 

In [ ]:
prediction=0
folds=12
repeats=15
epochsize=40
drop=0.5
kfold = KFold(n_splits=folds, shuffle=True)

for k in range(repeats):
  for train_ind, test_ind in kfold.split(X, y):
    model = Sequential()
    model.add(Conv1D (kernel_size=60, filters=256, strides=3, padding='valid',
                    kernel_initializer='he_uniform', input_shape=[600,7],
                    activation='relu'))
    model.add(MaxPooling1D(1, padding = 'valid'))

    model.add(Conv1D(kernel_size=60, filters=128, activation='relu'))
    model.add(MaxPooling1D(1, padding = 'valid'))

    model.add(Conv1D(kernel_size=60, filters=64, activation='relu'))
    model.add(MaxPooling1D(1, padding = 'valid'))

    model.add(BatchNormalization())

    model.add(Dropout(drop))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())

    model.add(Dropout(drop))
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())

    model.add(Dropout(drop))
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())


    model.add(Dropout(drop))
    model.add(Dense(61))
    model.add(Activation('softmax'))

    adam = optimizers.Adam(lr=0.0001)

    model.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])
    
    model.fit(np.array(X)[train_ind], y[train_ind], epochs=epochsize, batch_size=256,
              validation_data=(np.array(X)[test_ind], y[test_ind]))

    prediction += (model.predict(test_X)/(folds*repeats))

submission.iloc[:,1:]=prediction
submission.to_csv('/content/drive/MyDrive/월간데이콘11/py_answer1.csv', index=False)

In [ ]:
prediction=0
folds=12
repeats=15
epochsize=40
drop=0.5
kfold = KFold(n_splits=folds, shuffle=True)

for k in range(repeats):
  for train_ind, test_ind in kfold.split(X, y):
    model = Sequential()
    model.add(Conv1D (kernel_size=60, filters=256, strides=3, padding='valid',
                     kernel_initializer='he_uniform', input_shape=[600,7],
                     activation='relu'))
    model.add(MaxPooling1D(1, padding = 'valid'))

    model.add(Conv1D(kernel_size=60, filters=128, activation='relu'))
    model.add(MaxPooling1D(1, padding = 'valid'))

    model.add(BatchNormalization())

    model.add(Dropout(drop))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())

    model.add(Dropout(drop))
    model.add(Dense(61))
    model.add(Activation('softmax'))

    adam = optimizers.Adam(lr=0.0001)

    model.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])
  
    model.fit(np.array(X)[train_ind], y[train_ind], epochs=epochsize, batch_size=64,
              validation_data=(np.array(X)[test_ind], y[test_ind]))

    prediction += (model.predict(test_X)/(folds*repeats))

submission.iloc[:,1:]=prediction
submission.to_csv('/content/drive/MyDrive/월간데이콘11/py_answer2.csv', index=False)

In [ ]:
prediction=0
folds=12
repeats=15
epochsize=40
drop=0.5
kfold = KFold(n_splits=folds, shuffle=True)

for k in range(repeats):
  for train_ind, test_ind in kfold.split(X, y):
    model = Sequential()
    model.add(Conv1D (kernel_size=60, filters=256, strides=3, padding='valid',
                     kernel_initializer='he_uniform', input_shape=[600,7],
                     activation='relu'))
    model.add(MaxPooling1D(1, padding = 'valid'))
    
    model.add(Conv1D(kernel_size=60, filters=128, activation='relu'))
    model.add(MaxPooling1D(1, padding = 'valid'))
    
    model.add(Conv1D(kernel_size=60, filters=64, activation='relu'))
    model.add(MaxPooling1D(1, padding = 'valid'))

    model.add(Conv1D(kernel_size=60, filters=32, activation='relu'))
    model.add(MaxPooling1D(1, padding = 'valid'))

    model.add(BatchNormalization())

    model.add(Dropout(drop))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())

    model.add(Dropout(drop))
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())

    model.add(Dropout(drop))
    model.add(Dense(61))
    model.add(Activation('softmax'))

    adam = optimizers.Adam(lr=0.0001)

    model.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])
  
    model.fit(np.array(X)[train_ind], y[train_ind], epochs=epochsize, batch_size=64,
              validation_data=(np.array(X)[test_ind], y[test_ind]))

    prediction += (model.predict(test_X)/(folds*repeats))

submission.iloc[:,1:]=prediction
submission.to_csv('/content/drive/MyDrive/월간데이콘11/py_answer3.csv', index=False)

# Softvoting

In [ ]:
model1ans=pd.read_csv('/content/drive/MyDrive/월간데이콘11/py_answer1.csv')
model2ans=pd.read_csv('/content/drive/MyDrive/월간데이콘11/py_answer2.csv')
model3ans=pd.read_csv('/content/drive/MyDrive/월간데이콘11/py_answer3.csv')

In [ ]:
final_sv=(model1ans+model2ans+model3ans)/3

In [ ]:
# final_sv.to_csv('/content/drive/MyDrive/월간데이콘11/py_new4.csv', index=False)

final_sv.to_csv('/content/drive/MyDrive/월간데이콘11/py_final_answer.csv', index=False)